#Regression and Regularization

Using regularization - Lasso and Ridge

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.metrics import mean_squared_error, root_mean_squared_error
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
%matplotlib inline



We are going to use the `Credit` data from ISLP, which can be [downloaded from this link](https://drive.google.com/uc?download&id=1joK1gnatsAANBNBXCBk1vtEr1g3FwR2W)

In [20]:
# read file from disc
from google.colab import files
uploaded = files.upload()

In [21]:
# download Credit data from ISLP

Credit = pd.read_csv('Credit.csv')
Credit.head()


,ID,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
0,1,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333
1,2,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903
2,3,104.593,7075,514,4,71,11,Male,No,No,Asian,580
3,4,148.924,9504,681,3,36,11,Female,No,No,Asian,964
4,5,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331


This is a data set regarding credit card customers.  We want to see if we can predict how much balance they will keep on their cards based on demographics and other features.

In [ ]:
Credit.describe()

In [ ]:
Credit.hist(figsize=(15,15))
plt.show()

Features all here seem to be fairly well behaved, and no missing values.  

Because we are doing regression, we need to `get_dummies` for the categoricals (make sure to use `dtype=int` because regression algorithms need numerics.

In [22]:
Credit=pd.get_dummies(Credit,drop_first=True, dtype=int)
Credit.head()

,ID,Income,Limit,Rating,Cards,Age,Education,Balance,Gender_Female,Student_Yes,Married_Yes,Ethnicity_Asian,Ethnicity_Caucasian
0,1,14.891,3606,283,2,34,11,333,0,0,1,0,1
1,2,106.025,6645,483,3,82,15,903,1,1,1,1,0
2,3,104.593,7075,514,4,71,11,580,0,0,0,1,0
3,4,148.924,9504,681,3,36,11,964,1,0,0,1,0
4,5,55.882,4897,357,2,68,16,331,0,0,1,0,1


In [23]:
# define our X and y

X=Credit.drop(["ID","Balance"],axis=1)
y=Credit["Balance"]

First fit a standard OLS model

I like the `OLS` module from `statsmodels.api` because it provides a nice regression output table, but `sklearn.linear_model import LinearRegression` would work fine here also.

(OLS=_Ordinary Least Squares_ and is equivalent to Multiple Linear Regression)

In [24]:
from statsmodels.api import OLS

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

# unfortunately, it requires you to add a constant feature to run a regression
X_train = sm.add_constant(X_train)
mlr_model=OLS(y_train,X_train).fit()

print(mlr_model.summary(slim=True))

                            OLS Regression Results                            
Dep. Variable:                Balance   R-squared:                       0.954
Model:                            OLS   Adj. R-squared:                  0.952
No. Observations:                 320   F-statistic:                     582.3
Covariance Type:            nonrobust   Prob (F-statistic):          9.05e-199
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                -490.9058     39.263    -12.503      0.000    -568.163    -413.648
Income                 -7.6735      0.269    -28.553      0.000      -8.202      -7.145
Limit                   0.1644      0.037      4.398      0.000       0.091       0.238
Rating                  1.4950      0.559      2.674      0.008       0.395       2.595
Cards                  16.2180      4.845      3.347      0.001       6.684  

Review the regression table...recall that those with p-value (or P>|t|) less than 0.05 are typically labelled as significant.  Which features are significant??

The table above is all based on the training set, lets calculate an RMSE on the test set.

In [25]:
# unfortunately, it requires you to add a constant feature to run a regression
X_test = sm.add_constant(X_test)
y_pred=mlr_model.predict(X_test)

mlr_rmse=root_mean_squared_error(y_pred,y_test)

print(round(mlr_rmse,1))


96.6


Can you interpret this RMSE value in the context of the problem?

## Ridge Regression

Now lets see if ridge regression can do any better.  Remember that Ridge with $\alpha=0$ is the same as regular multiple regression (OLS) and should match the value above.


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

alpha_best = 0
rmse_best = 10000

for a in np.arange(0,10,.5):
  model = Ridge(alpha=a, max_iter=10000) # what we called lambda in class is alpha in the Ridge function

  model.fit(X_train, y_train)
  y_pred=model.predict(X_test)
  rmse=root_mean_squared_error(y_pred,y_test)
  print("alpha=",a,": rmse = ",round(rmse,3))
  if rmse < rmse_best:
      rmse_best = rmse
      alpha_best = a
print("\nBest alpha = ",alpha_best)

alpha= 0.0 : rmse =  96.628
alpha= 0.5 : rmse =  96.926
alpha= 1.0 : rmse =  97.246
alpha= 1.5 : rmse =  97.585
alpha= 2.0 : rmse =  97.939
alpha= 2.5 : rmse =  98.306
alpha= 3.0 : rmse =  98.685
alpha= 3.5 : rmse =  99.073
alpha= 4.0 : rmse =  99.469
alpha= 4.5 : rmse =  99.871
alpha= 5.0 : rmse =  100.279
alpha= 5.5 : rmse =  100.69
alpha= 6.0 : rmse =  101.105
alpha= 6.5 : rmse =  101.521
alpha= 7.0 : rmse =  101.939
alpha= 7.5 : rmse =  102.357
alpha= 8.0 : rmse =  102.775
alpha= 8.5 : rmse =  103.192
alpha= 9.0 : rmse =  103.608
alpha= 9.5 : rmse =  104.022

Best alpha =  0.0


What is the best value of $\alpha$?  What does this tell you?

## Lasso Regression

Lasso regularized regression is the same as Ridge, but with a different penalty (L1) based on absolute values.  Can we do better?

In [28]:
alpha_best = 0
rmse_best = 10000

for a in np.arange(1,10,.5):
  model = Lasso(alpha=a, max_iter=10000) # what we called lambda in class is alpha in the Ridge function

  model.fit(X_train, y_train)
  y_pred=model.predict(X_test)
  rmse=root_mean_squared_error(y_pred,y_test)
  print("alpha=",a,": rmse = ",round(rmse,3))
  if rmse < rmse_best:
      rmse_best = rmse
      alpha_best = a
print("\nBest alpha = ",alpha_best)

alpha= 1.0 : rmse =  96.589
alpha= 1.5 : rmse =  96.533
alpha= 2.0 : rmse =  96.48
alpha= 2.5 : rmse =  96.459
alpha= 3.0 : rmse =  96.477
alpha= 3.5 : rmse =  96.535
alpha= 4.0 : rmse =  96.689
alpha= 4.5 : rmse =  96.873
alpha= 5.0 : rmse =  97.083
alpha= 5.5 : rmse =  97.393
alpha= 6.0 : rmse =  97.827
alpha= 6.5 : rmse =  98.28
alpha= 7.0 : rmse =  98.752
alpha= 7.5 : rmse =  99.243
alpha= 8.0 : rmse =  99.754
alpha= 8.5 : rmse =  100.282
alpha= 9.0 : rmse =  100.829
alpha= 9.5 : rmse =  101.393

Best alpha =  2.5


Looks like we might be able to do better with a little shrinkage!  What is the best across all methods?

Lets take this best method and look at the shrunken coefficients compared to the full (OLS) model.

In [29]:
model_noshrink = Lasso(alpha=0)
model_noshrink.fit(X_train,y_train)
model_noshrink.coef_

model_best = Lasso(alpha=alpha_best)
model_best.fit(X_train,y_train)
model_best.coef_

coef_table = zip(X.columns,model_best.coef_.round(4),model_noshrink.coef_.round(4))


coef_df = pd.DataFrame(coef_table, columns=['colname', 'coef_best','coef_noshrink'])

print(coef_df)

                colname  coef_best  coef_noshrink
0                Income    -7.6440        -7.6739
1                 Limit     0.1649         0.1658
2                Rating     1.4803         1.4746
3                 Cards    14.7318        16.3135
4                   Age    -0.4632        -0.4458
5             Education    -0.0000        -0.0901
6         Gender_Female    -4.4574       -15.2526
7           Student_Yes   391.3785       417.0731
8           Married_Yes    -9.7489       -20.1401
9       Ethnicity_Asian     0.0000        15.1068
10  Ethnicity_Caucasian     0.0000         8.8808


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.539e+06, tolerance: 6.708e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent

Note the coefficients that were shrunk ALL the way to zero.  This is one of the great features of the Lasso.  These are noise (useless) features and we basically are removing them by setting their values to zero.  

## Regularization with Logistic Regression

We will look again at the Tayko data from the Shmueli book: [download link here](https://drive.google.com/uc?download&id=1wo7x7PmnCJ5-79RZXJSIAa7eS8DdrX-y).  This is a company that is trying to predict Purchase from catalog mailings and other customer attributes.

The features consist of a bunch of indicator features for catalogs they have sent out to customers.  The target feature is "Purchase".



In [ ]:

from google.colab import files
uploaded = files.upload()


In [30]:
tayko = pd.read_csv('Tayko.csv')
tayko.info()
tayko.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   sequence_number       2000 non-null   int64
 1   US                    2000 non-null   int64
 2   source_a              2000 non-null   int64
 3   source_c              2000 non-null   int64
 4   source_b              2000 non-null   int64
 5   source_d              2000 non-null   int64
 6   source_e              2000 non-null   int64
 7   source_m              2000 non-null   int64
 8   source_o              2000 non-null   int64
 9   source_h              2000 non-null   int64
 10  source_r              2000 non-null   int64
 11  source_s              2000 non-null   int64
 12  source_t              2000 non-null   int64
 13  source_u              2000 non-null   int64
 14  source_p              2000 non-null   int64
 15  source_x              2000 non-null   int64
 16  source

,sequence_number,US,source_a,source_c,source_b,source_d,source_e,source_m,source_o,source_h,...,source_x,source_w,Freq,last_update_days_ago,1st_update_days_ago,Web order,Gender=male,Address_is_res,Purchase,Spending
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000
mean,1000.500000,0.824500,0.126500,0.056000,0.060000,0.041500,0.151000,0.01650,0.033500,0.052500,...,0.018000,0.137500,1.417000,2155.101000,2435.601500,0.426000,0.524500,0.221000,0.500000,102.62500
std,577.494589,0.380489,0.332495,0.229979,0.237546,0.199493,0.358138,0.12742,0.179983,0.223089,...,0.132984,0.344461,1.405738,1141.302846,1077.872233,0.494617,0.499524,0.415024,0.500125,186.78261
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,500.750000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,1.000000,1133.000000,1671.250000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,1000.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,1.000000,2280.000000,2721.000000,0.000000,1.000000,0.000000,0.500000,2.00000
75%,1500.250000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,2.000000,3139.250000,3353.000000,1.000000,1.000000,0.000000,1.000000,153.00000
max,2000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,...,1.000000,1.000000,15.000000,4188.000000,4188.000000,1.000000,1.000000,1.000000,1.000000,1500.00000


In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

X=tayko.drop(["sequence_number","Spending","Purchase"],axis=1)
# add constant term to X
X = sm.add_constant(X)
y=tayko["Purchase"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)



In [32]:
## fit FULL model - Logistic Regression.

## Use statmodels so we can get the full logistic regression table.
logit_model = sm.Logit(y_train, X_train).fit()

# Print the full coefficients table
print(logit_model.summary())


y_pred_prob = logit_model.predict(X_test)

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_prob)

print("\nAUC:", round(auc,4))



Optimization terminated successfully.
         Current function value: 0.381030
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:               Purchase   No. Observations:                 1600
Model:                          Logit   Df Residuals:                     1577
Method:                           MLE   Df Model:                           22
Date:                Wed, 12 Mar 2025   Pseudo R-squ.:                  0.4503
Time:                        15:17:52   Log-Likelihood:                -609.65
converged:                       True   LL-Null:                       -1109.0
Covariance Type:            nonrobust   LLR p-value:                3.788e-197
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -4.8199      0.660     -7.298      0.000      -6.114      -3.525

In [35]:
#. Now we want to fit a regularlized model - lasso/ridge

## We need to switch to the LogisticRegression() function from sklearn, because statsmodels does not have the lasso.

## For logistic regression the C parameter is 1/alpha

c_best = 0
auc_best = 0

# you need different ranges for lasso and ridge...

for c in np.arange(.1,10,.1):
  model = LogisticRegression(penalty='l1',C=c,solver='liblinear') # what we called lambda in class is alpha in the Ridge function
  model.fit(X_train, y_train)
  y_pred=model.predict_proba(X_test)[:, 1]
#  rmse=root_mean_squared_error(y_pred,y_test)
  auc = roc_auc_score(y_test, y_pred)
 # print("C=",round(c,2),"auc=",round(auc,4))
  if auc > auc_best:
      auc_best = auc
      c_best = c
print("\nBest c = ",c_best, round(auc_best,4))



Best c =  1.1 0.892


Can toggle between Ridge and Lasso by changing the `penalty` from `l1` to `l2`

Now lets build the table to see if lasso removes any features

In [36]:
model_noshrink = LogisticRegression(penalty='l1',C=10000,solver='liblinear')
model_noshrink.fit(X_train,y_train)

c_best=1.1

model_best = LogisticRegression(penalty='l1',C=c_best,solver='liblinear')
model_best.fit(X_train,y_train)

pd.DataFrame(zip(X.columns, model_best.coef_[0].round(3), model_noshrink.coef_[0].round(3)),
             columns=['Feature', 'Coefficient_best','Coefficient_noshrink'])


,Feature,Coefficient_best,Coefficient_noshrink
0,const,-1.203,-2.745
1,US,0.000,0.059
2,source_a,1.087,2.722
3,source_c,-0.856,0.607
4,source_b,-1.276,0.148
5,source_d,0.000,1.472
6,source_e,0.028,1.644
7,source_m,0.430,2.237
8,source_o,0.000,1.802
9,source_h,-3.969,-2.625


In [37]:
### Now lets try stepwise!

# Start with all features
features = list(X_train.columns)
best_auc = .888
best_features = features.copy()

while len(features) > 0:
        # Fit model with current features
        model = sm.Logit(y_train, X_train[best_features]).fit(disp=0) # Use statsmodels for p-values
        y_pred = model.predict(X_test[best_features])
        auc = roc_auc_score(y_test, y_pred)

        # Find feature with highest p-value
        p_values = model.pvalues # Exclude intercept
        worst_feature_index = np.argmax(p_values)
        worst_feature = features[worst_feature_index]
        print(f"Worst feature: {worst_feature}")

        # fit model with feature removed, and check AUC
        features.remove(worst_feature)
        new_features = features.copy()
        model_new = sm.Logit(y_train, X_train[new_features]).fit(disp=0)
        y_pred_new = model_new.predict(X_test[new_features])
        new_auc = roc_auc_score(y_test, y_pred_new)
        print("New AUC" ,round(new_auc,3), round(best_auc,3))
        if round(new_auc,3) >= round(best_auc,3):
            print ("REMOVE",worst_feature,round(new_auc,3),"\n")
            best_features = new_features.copy()
            best_auc = new_auc
        else:
            print ("Stop, do not remove",worst_feature)
            break  # Stop if no more features to remove


print("\nSelected Features:", best_features)
print("Best AUC:", best_auc)


Worst feature: source_b
New AUC 0.888 0.888
REMOVE source_b 0.888 

Worst feature: US
New AUC 0.888 0.888
REMOVE US 0.888 

Worst feature: Gender=male
New AUC 0.888 0.888
REMOVE Gender=male 0.888 

Worst feature: last_update_days_ago
New AUC 0.888 0.888
REMOVE last_update_days_ago 0.888 

Worst feature: source_c
New AUC 0.889 0.888
REMOVE source_c 0.889 

Worst feature: 1st_update_days_ago
New AUC 0.89 0.889
REMOVE 1st_update_days_ago 0.89 

Worst feature: source_o
New AUC 0.895 0.89
REMOVE source_o 0.895 

Worst feature: source_p
New AUC 0.894 0.895
Stop, do not remove source_p

Selected Features: ['const', 'source_a', 'source_d', 'source_e', 'source_m', 'source_h', 'source_r', 'source_s', 'source_t', 'source_u', 'source_p', 'source_x', 'source_w', 'Freq', 'Web order', 'Address_is_res']
Best AUC: 0.894758579259593


In [38]:
# Fit final model with selected features
final_model = sm.Logit(y_train, X_train[best_features]).fit(disp=0)

print(final_model.summary())
# Evaluate on test set
y_pred_final = final_model.predict(X_test[best_features])
auc_final = roc_auc_score(y_test, y_pred_final)
print("Final AUC:", round(auc_final,4))
# print("Final RMSE on Test Set:", rmse_final)

                           Logit Regression Results                           
Dep. Variable:               Purchase   No. Observations:                 1600
Model:                          Logit   Df Residuals:                     1584
Method:                           MLE   Df Model:                           15
Date:                Wed, 12 Mar 2025   Pseudo R-squ.:                  0.4462
Time:                        15:30:01   Log-Likelihood:                -614.16
converged:                       True   LL-Null:                       -1109.0
Covariance Type:            nonrobust   LLR p-value:                2.248e-201
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -4.4327      0.284    -15.583      0.000      -4.990      -3.875
source_a           2.0785      0.289      7.193      0.000       1.512       2.645
source_d           0.8426      0.387